<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/err/angel_one_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# --- Colab-ready extractor for Angel One Scrip Master JSON ---

import json, io, os, re
from typing import Optional
import pandas as pd

def load_scrip_master(src: str) -> pd.DataFrame:
    """
    Load OpenAPIScripMaster.json from a URL or local path and return a normalized DataFrame
    with columns: tradingsymbol, symboltoken, exchange, instrumenttype, name, expiry, strike, lotsize.
    """
    # detect URL vs path
    if re.match(r"^https?://", src):
        import requests
        r = requests.get(src, timeout=30)
        r.raise_for_status()
        raw = r.json()
    else:
        with open(src, "r", encoding="utf-8") as f:
            raw = json.load(f)

    # Some dumps wrap the list under {"data": [...]}
    records = raw.get("data") if isinstance(raw, dict) else raw
    if not isinstance(records, list):
        raise ValueError("Unexpected JSON structure: expected a list at top-level or under 'data'.")

    # Normalize records into a DataFrame
    def norm(rec: dict) -> dict:
        r = {str(k).lower(): v for k, v in rec.items()}
        return {
            "tradingsymbol": r.get("tradingsymbol") or r.get("symbol"),
            "symboltoken": r.get("symboltoken") or r.get("token"),
            "exchange": r.get("exch_seg") or r.get("exchange") or r.get("exch"),
            "instrumenttype": r.get("instrumenttype"),
            "name": r.get("name"),
            "expiry": r.get("expiry"),
            "strike": r.get("strike"),
            "lotsize": r.get("lotsize"),
        }

    df = pd.DataFrame([norm(x) for x in records])
    # nice types
    for col in ["strike", "lotsize"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

def filter_symbols(
    df: pd.DataFrame,
    exchange: Optional[str] = None,
    instrumenttype: Optional[str] = None,
    query: Optional[str] = None,
) -> pd.DataFrame:
    """
    Filter by exchange (e.g., NSE/NFO/BSE/MCX), instrumenttype (e.g., EQ/FUTIDX/OPTIDX),
    and a free-text query (matches tradingsymbol or name).
    """
    out = df.copy()
    if exchange:
        out = out[out["exchange"].str.upper() == exchange.upper()]
    if instrumenttype:
        out = out[out["instrumenttype"].str.upper() == instrumenttype.upper()]
    if query:
        q = query.lower()
        out = out[
            out["tradingsymbol"].fillna("").str.lower().str.contains(q)
            | out["name"].fillna("").str.lower().str.contains(q)
        ]
    return out.sort_values(["exchange", "instrumenttype", "tradingsymbol"], na_position="last").reset_index(drop=True)

# ------------------ EXAMPLES (run these as-is) ------------------

SRC = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json?_gl=1*1quceu5*_gcl_au*MTAzODcwNzA1NC4xNzUwNDMzOTg2*_ga*OTM2NDUyNzg3LjE3MjAwNzIyMjg.*_ga_CDX93S7LDP*czE3NTQ5MjYxOTMkbzg3JGcxJHQxNzU0OTI3MDA2JGo2MCRsMCRoMA.."




Loaded rows: 124,700


,tradingsymbol,symboltoken,exchange,instrumenttype,name,expiry,strike,lotsize
0,Nifty 50,99926000,NSE,AMXIDX,NIFTY,,0.0,1
1,Nifty GrowSect 15,99926001,NSE,AMXIDX,NIFTY GROWSECT 15,,0.0,1
2,Nifty50 PR 2x Lev,99926002,NSE,AMXIDX,NIFTY50 PR 2X LEV,,0.0,1
3,Nifty 500,99926004,NSE,AMXIDX,NIFTY 500,,0.0,1
4,Nifty IT,99926008,NSE,AMXIDX,NIFTY IT,,0.0,1
5,Nifty Bank,99926009,NSE,AMXIDX,BANKNIFTY,,0.0,1
6,NIFTY MIDCAP 100,99926011,NSE,AMXIDX,NIFTY MIDCAP 100,,0.0,1
7,Nifty 100,99926012,NSE,AMXIDX,NIFTY 100,,0.0,1
8,Nifty Next 50,99926013,NSE,AMXIDX,NIFTYNXT50,,0.0,1
9,Nifty Midcap 50,99926014,NSE,AMXIDX,NIFTY MIDCAP 50,,0.0,1


In [ ]:
df = load_scrip_master(SRC)
print(f"Loaded rows: {len(df):,}")
df.head(10)

In [4]:
# Find NIFTY futures (NFO / FUTIDX)
nifty_fut = filter_symbols(df, exchange="NFO", instrumenttype="FUTIDX", query="NIFTY")
nifty_fut[["tradingsymbol", "symboltoken", "exchange", "instrumenttype", "expiry", "strike", "lotsize"]].head(20)


,tradingsymbol,symboltoken,exchange,instrumenttype,expiry,strike,lotsize
0,BANKNIFTY28AUG25FUT,64100,NFO,FUTIDX,28AUG2025,-1.0,35
1,BANKNIFTY28OCT25FUT,52145,NFO,FUTIDX,28OCT2025,-1.0,35
2,BANKNIFTY30SEP25FUT,52995,NFO,FUTIDX,30SEP2025,-1.0,35
3,FINNIFTY28AUG25FUT,64101,NFO,FUTIDX,28AUG2025,-1.0,65
4,FINNIFTY28OCT25FUT,52146,NFO,FUTIDX,28OCT2025,-1.0,65
5,FINNIFTY30SEP25FUT,52997,NFO,FUTIDX,30SEP2025,-1.0,65
6,MIDCPNIFTY28AUG25FUT,64102,NFO,FUTIDX,28AUG2025,-1.0,140
7,MIDCPNIFTY28OCT25FUT,52147,NFO,FUTIDX,28OCT2025,-1.0,140
8,MIDCPNIFTY30SEP25FUT,52998,NFO,FUTIDX,30SEP2025,-1.0,140
9,NIFTY28AUG25FUT,64103,NFO,FUTIDX,28AUG2025,-1.0,75
